In [1]:
from pymongo import MongoClient
import pandas as pd  
import numpy as np 
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from nltk.corpus import stopwords

In [2]:
client = MongoClient('localhost', 27017)
db = client['twitter']

pipeline=[{ "$project": {  "id_user" : 1, "originalTweet" : 1, "hashtag" : 1, "mention" : 1} }]

cursor_list = list(db['tweets'].aggregate(pipeline))

In [3]:
tweets = pd.DataFrame(cursor_list)
data = pd.DataFrame(columns=['Tweets'])
data['Tweets'] = tweets['originalTweet']
data['user_id'] = tweets['id_user']
data['len']  = np.array([len(tweet) for tweet in data['Tweets']])
data['hahtags'] = tweets['hashtag']
data['mentions'] = tweets['mention']

In [4]:
# data = pd.DataFrame(columns=['Tweets','Id','len','Hashtags'])
# data['Tweets'] = tweets['text']
# data['Id'] = tweets['_id']
# data['len']  = np.array([len(tweet) for tweet in data['Tweets']])
# user = list()
# for i in range(0,len(tweets)):
#     user.append(tweets['user'][i]['name'])
# data['user'] = np.array(user)

# for i in range(0,len(tweets)):
#     hashtags= list()
#     if (len(tweets['entities'][i]['hashtags'])==0):
#         hashtags.append('NaN')
#     else:
#         for n in range(len(tweets['entities'][i]['hashtags'])):
#             hashtags.append(tweets['entities'][i]['hashtags'][n]['text'])
#             data.at[i, 'Hashtags'] = hashtags          

In [ ]:
data

In [6]:
filtered_data_hashtags = data.hahtags.dropna()

In [ ]:
# n = filtered_data.reset_index(drop=True)
# n.head(20)

In [ ]:
# n['liststring'] = [','.join(map(str, l)) for l in n.Hashtags]
# n.head(20)

In [9]:
vectorizer = TfidfVectorizer(use_idf=True,ngram_range=(1,3))

X = vectorizer.fit_transform(filtered_data_hashtags)

true_k = 10
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster :", i),
    for ind in order_centroids[i, :5]:
        print(' ', terms[ind]),
    print

Top terms per cluster:
Cluster : 0
  p2
  tcot
  p2 tcot
  tcot uniteblue
  p2 tcot uniteblue
Cluster : 1
  rhobh
  rhobh rhobh
  funcionbeneficio subastabeneficio
  función felizdomingo
  funcomfortable
Cluster : 2
  parenting
  kids
  parenting kids
  parenting kids home
  kids home
Cluster : 3
  brexit
  brexit brexit
  brexit eu brexit
  brexit eu
  eu brexit
Cluster : 4
  hongkong
  china
  hongkong china
  china hongkong
  occupycentral
Cluster : 5
  periscope
  periscope oneonone
  prophet
  oneonone
  prophet god
Cluster : 6
  quote
  quote quote
  zürichwest
  functionredblend
  funcomfortable
Cluster : 7
  mplusrewards
  breaking
  ff
  trump
  debate
Cluster : 8
  chattanooga
  chanews
  chattanooga chanews
  chanews chattanooga
  tennessee
Cluster : 9
  fb
  breakingnews
  breakingnews fb
  fb fb
  zürichwest


In [10]:
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import collections
plt.bar(range(len(set(model.labels_))), np.bincount(model.labels_))

plt.ylabel('population')
plt.xlabel('cluster label')
plt.title('population sizes with {} clusters'.format(true_k));

# truncate y axis to see the rest better
plt.ylim(0,10000);


print (collections.Counter(model.labels_))
print ('Silouette_score: ',silhouette_score(X, model.labels_))

Counter({7: 1249905, 6: 10447, 9: 8705, 0: 5367, 3: 3109, 5: 2112, 8: 1613, 1: 329, 4: 250, 2: 209})


MemoryError: 